# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [6]:
df[0]

'03/25/93 Total time of visit (in minutes):\n'

In [9]:
df.size

500

In [104]:
# strip the newline symbol at the end of every line
for i in range(df.size):
    df[i] = df[i].strip()
    #print (df[i])

In [186]:
def date_sorter():
    import re
    from calendar import month_name
    import dateutil.parser
    from datetime import datetime

    df = pd.DataFrame(doc, columns=['text'])
    #print(df)
    pattern = "[,.]? \d{4}|".join(month_name[1:]) + "[,.]? \d{4}"
    print(pattern)
    #['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    
    df['text'] = df['text'].apply(lambda x: x.strip('\n'))
    
    df['date'] = df['text'].apply(lambda x:re.findall(r'\d{1,2}\/\d{1,2}\/\d{2,4}|\d{1,2}\-\d{1,2}\-\d{2,4}|[A-Z][a-z]+\-\d{1,2}\-\d{4}|[A-Z][a-z]+[,.]? \d{2}[a-z]*,? \d{4}|\d{1,2} [A-Z][a-z,.]+ \d{4}|[A-Z][a-z]{2}[,.]? \d{4}|'+pattern+r'|\d{1,2}\/\d{4}|\d{4}',x))
    # there are few wrong dates, we correct them one by one
    df['date'][271] = [df['date'][271][1]]
    df['date'] = df['date'].apply(lambda x: x[0])
    df['date'][461] = re.findall(r'\d{4}',df['date'][461])[0]
    df['date'][465] = re.findall(r'\d{4}',df['date'][465])[0]
    
    date_list = list(df['date'])
    
    for i in range(len(date_list)):
        if(re.match(r'\d{4}',date_list[i])) :
            #print(year)
            date_list[i] = 'January 1, '+date_list[i]
        elif (re.match(r'\d{1,2}\/\d{4}',date_list[i])) :
            date_split = date_list[i].split('/')
            date_list[i] = date_split[0] + '/1/'+date_split[1]
        elif(re.match(r'[A-Z][a-z]+[,.]? \d{4}',date_list[i])) :
            date_split = date_list[i].split(' ')
            date_list[i] = date_split[0] + ' 1 '+date_split[1]
        date_list[i] = dateutil.parser.parse(date_list[i]).strftime("%m/%d/%Y")
    
    df['date'] = date_list
    #print(df)
    #df['index'] = df['date'].rank().subtract(1)
    #print(min(df['index']))
    fun = lambda date: datetime.strptime(date, "%m/%d/%Y")
    df['index'] = sorted(range(len(date_list)), key=lambda x : fun(date_list[x]))
    sdf = df
    sdf.drop('text', axis=1,inplace=True)
    #result = sdf.sort_values(by='date',axis=0, ascending=False, kind='mergesort')
    final = list(sdf['index'])
    final_series = pd.Series(final)    
    return final_series

In [180]:
df

0             03/25/93 Total time of visit (in minutes):
1                           6/18/85 Primary Care Doctor:
2      sshe plans to move as of 7/8/71 In-Home Servic...
3                    7 on 9/27/75 Audit C Score Current:
4      2/6/96 sleep studyPain Treatment Pain Level (N...
                             ...                        
495    1979 Family Psych History: Family History of S...
496    therapist and friend died in ~2006 Parental/Ca...
497                           2008 partial thyroidectomy
498    sPt describes a history of sexual abuse as a c...
499    . In 1980, patient was living in Naples and de...
Length: 500, dtype: object

In [189]:
for line in df: 
    print(re.findall(r'\d{1,2}\/\d{1,2}\/\d{2,4}',line))
    #print(re.findall(r'\d{1,2}\-\d{1,2}\-\d{2,4}',line))
    #print(re.findall('[A-Z][a-z]+[,.]? \d{2}[a-z]*,? \d{4}',line))
    #print(re.findall(r'\d{1,2} [A-Z][a-z,.]+ \d{4}',line))
    #print(re.findall(r'[A-Z][a-z]{2}[,.]? \d{4}',line))
    #print(re.findall(r'[A-Z][a-z]{2}[,.]? \d{4}',line))

['03/25/93']
['6/18/85']
['7/8/71']
['9/27/75']
['2/6/96']
['7/06/79']
['5/18/78']
['10/24/89']
['3/7/86']
['4/10/71']
['5/11/85']
['4/09/75']
['8/01/98']
['1/26/72']
['5/24/1990']
['1/25/2011']
['4/12/82']
['10/13/1976']
['4/24/98']
['5/21/77']
['7/21/98']
['10/21/79']
['3/03/90']
['2/11/76']
['07/25/1984']
[]
['9/22/89']
['9/02/76']
['9/12/71']
['10/24/86']
['03/31/1985']
['7/20/72']
['4/12/87']
['06/20/91']
['5/12/2012']
['3/15/83']
['2/14/73']
['5/24/88']
['7/27/1986']
[]
[]
['6/24/87']
['8/14/94']
['4/13/2002']
['8/16/82']
['2/15/1998']
['7/15/91']
['06/12/94']
['9/17/84']
['2/28/75']
['11/22/75']
['5/24/91']
['6/13/92']
['7/11/71']
['12/26/86']
['10/11/1987']
['3/14/95']
['12/01/73']
['12/5/2010']
['08/20/1982']
['7/24/95']
['8/06/83']
['02/22/92']
['6/28/87']
['07/29/1994']
['08/11/78']
['10/29/91']
['7/6/91']
['1/21/87']
['11/3/1985']
['7/04/82']
[]
['7/11/77', '9/36/308']
['4/12/74']
['09/19/81']
['9/6/79']
['12/5/87']
['01/05/1999']
['4/22/80']
['10/04/98']
['6/29/81']
['8/04

In [187]:
date_sorter()

January[,.]? \d{4}|February[,.]? \d{4}|March[,.]? \d{4}|April[,.]? \d{4}|May[,.]? \d{4}|June[,.]? \d{4}|July[,.]? \d{4}|August[,.]? \d{4}|September[,.]? \d{4}|October[,.]? \d{4}|November[,.]? \d{4}|December[,.]? \d{4}


0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64

In [116]:
"a".join(['b','c','d'])

'bacad'